In [11]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Evaluation.K_FoldEvaluator import K_FoldEvaluator
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [12]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()

URM_train_list=[]
URM_validation_list=[]
evaluator_list=[]

In [13]:
for k in range(5):
    URM_train,URM_validation = split_train_in_two_percentage_global_sample(URM, train_percentage=0.8)
    URM_train_list.append(URM_train)
    URM_validation_list.append(URM_validation)
    evaluator_list.append(EvaluatorHoldout(URM_validation, cutoff_list=[10],verbose=False))

In [14]:
URM_combined_list=[]
for URM_i in URM_train_list:
    URM_combined_list.append(combine_matrices(ICM=ICM,URM=URM_i))

In [15]:
from bayes_opt import BayesianOptimization

In [16]:
IALS_recommenders=[]


for i in range(len(URM_train_list)):
    
    IALS_recommenders.append(IALSRecommender(URM_train=URM_combined_list[i]))

In [17]:
tuning_params = {
    "epochs": (5,60),
    "num_factors": (20,60),
    "alpha": (0.1,1.0),
    "reg": (0.1,5)
}

In [18]:

def BO_function(epochs,num_factors,alpha,reg):
    MAP = 0
    k = 5
    cutoff=10
    for i in range(len(IALS_recommenders)):
        IALS_recommenders[i].fit(epochs=int(epochs),num_factors=int(num_factors), alpha=alpha,reg=reg)
        result_df, _ = evaluator_list[i].evaluateRecommender(IALS_recommenders[i])
        MAP += result_df.loc[cutoff]["MAP"]
        
    return MAP/k


In [19]:
optimizer = BayesianOptimization(f=BO_function,
                                pbounds=tuning_params,
                                verbose=5,
                                random_state=5)

optimizer.maximize(init_points=80,n_iter=70)


|   iter    |  target   |   alpha   |  epochs   | num_fa... |    reg    |
-------------------------------------------------------------------------
IALSRecommenderLinear: Epoch 1 of 52. Elapsed time 7.21 sec
IALSRecommenderLinear: Epoch 2 of 52. Elapsed time 15.01 sec
IALSRecommenderLinear: Epoch 3 of 52. Elapsed time 19.62 sec
IALSRecommenderLinear: Epoch 4 of 52. Elapsed time 24.30 sec
IALSRecommenderLinear: Epoch 5 of 52. Elapsed time 28.92 sec
IALSRecommenderLinear: Epoch 6 of 52. Elapsed time 33.49 sec
IALSRecommenderLinear: Epoch 7 of 52. Elapsed time 38.38 sec
IALSRecommenderLinear: Epoch 8 of 52. Elapsed time 42.89 sec
IALSRecommenderLinear: Epoch 9 of 52. Elapsed time 47.09 sec
IALSRecommenderLinear: Epoch 10 of 52. Elapsed time 51.79 sec
IALSRecommenderLinear: Epoch 11 of 52. Elapsed time 56.37 sec
IALSRecommenderLinear: Epoch 12 of 52. Elapsed time 1.01 min
IALSRecommenderLinear: Epoch 13 of 52. Elapsed time 1.08 min
IALSRecommenderLinear: Epoch 14 of 52. Elapsed time 1.15 m

IALSRecommenderLinear: Epoch 27 of 52. Elapsed time 3.28 min
IALSRecommenderLinear: Epoch 28 of 52. Elapsed time 3.40 min
IALSRecommenderLinear: Epoch 29 of 52. Elapsed time 3.51 min
IALSRecommenderLinear: Epoch 30 of 52. Elapsed time 3.63 min
IALSRecommenderLinear: Epoch 31 of 52. Elapsed time 3.75 min
IALSRecommenderLinear: Epoch 32 of 52. Elapsed time 3.87 min
IALSRecommenderLinear: Epoch 33 of 52. Elapsed time 3.99 min
IALSRecommenderLinear: Epoch 34 of 52. Elapsed time 4.10 min
IALSRecommenderLinear: Epoch 35 of 52. Elapsed time 4.22 min
IALSRecommenderLinear: Epoch 36 of 52. Elapsed time 4.34 min
IALSRecommenderLinear: Epoch 37 of 52. Elapsed time 4.45 min
IALSRecommenderLinear: Epoch 38 of 52. Elapsed time 4.57 min
IALSRecommenderLinear: Epoch 39 of 52. Elapsed time 4.68 min
IALSRecommenderLinear: Epoch 40 of 52. Elapsed time 4.80 min
IALSRecommenderLinear: Epoch 41 of 52. Elapsed time 4.92 min
IALSRecommenderLinear: Epoch 42 of 52. Elapsed time 5.04 min
IALSRecommenderLinear: E

IALSRecommenderLinear: Epoch 1 of 38. Elapsed time 18.65 sec
IALSRecommenderLinear: Epoch 2 of 38. Elapsed time 35.79 sec
IALSRecommenderLinear: Epoch 3 of 38. Elapsed time 53.92 sec
IALSRecommenderLinear: Epoch 4 of 38. Elapsed time 1.16 min
IALSRecommenderLinear: Epoch 5 of 38. Elapsed time 1.43 min
IALSRecommenderLinear: Epoch 6 of 38. Elapsed time 1.68 min
IALSRecommenderLinear: Epoch 7 of 38. Elapsed time 1.94 min
IALSRecommenderLinear: Epoch 8 of 38. Elapsed time 2.20 min
IALSRecommenderLinear: Epoch 9 of 38. Elapsed time 2.44 min
IALSRecommenderLinear: Epoch 10 of 38. Elapsed time 2.69 min
IALSRecommenderLinear: Epoch 11 of 38. Elapsed time 2.96 min
IALSRecommenderLinear: Epoch 12 of 38. Elapsed time 3.21 min
IALSRecommenderLinear: Epoch 13 of 38. Elapsed time 3.44 min
IALSRecommenderLinear: Epoch 14 of 38. Elapsed time 3.71 min
IALSRecommenderLinear: Epoch 15 of 38. Elapsed time 3.97 min
IALSRecommenderLinear: Epoch 16 of 38. Elapsed time 4.23 min
IALSRecommenderLinear: Epoch 1

IALSRecommenderLinear: Epoch 19 of 38. Elapsed time 4.42 min
IALSRecommenderLinear: Epoch 20 of 38. Elapsed time 4.56 min
IALSRecommenderLinear: Epoch 21 of 38. Elapsed time 4.69 min
IALSRecommenderLinear: Epoch 22 of 38. Elapsed time 4.82 min
IALSRecommenderLinear: Epoch 23 of 38. Elapsed time 4.95 min
IALSRecommenderLinear: Epoch 24 of 38. Elapsed time 5.08 min
IALSRecommenderLinear: Epoch 25 of 38. Elapsed time 5.20 min
IALSRecommenderLinear: Epoch 26 of 38. Elapsed time 5.33 min
IALSRecommenderLinear: Epoch 27 of 38. Elapsed time 5.46 min
IALSRecommenderLinear: Epoch 28 of 38. Elapsed time 5.58 min
IALSRecommenderLinear: Epoch 29 of 38. Elapsed time 5.71 min
IALSRecommenderLinear: Epoch 30 of 38. Elapsed time 5.84 min
IALSRecommenderLinear: Epoch 31 of 38. Elapsed time 5.96 min
IALSRecommenderLinear: Epoch 32 of 38. Elapsed time 6.09 min
IALSRecommenderLinear: Epoch 33 of 38. Elapsed time 6.21 min
IALSRecommenderLinear: Epoch 34 of 38. Elapsed time 6.35 min
IALSRecommenderLinear: E

IALSRecommenderLinear: Epoch 9 of 15. Elapsed time 37.65 sec
IALSRecommenderLinear: Epoch 10 of 15. Elapsed time 41.87 sec
IALSRecommenderLinear: Epoch 11 of 15. Elapsed time 46.14 sec
IALSRecommenderLinear: Epoch 12 of 15. Elapsed time 50.27 sec
IALSRecommenderLinear: Epoch 13 of 15. Elapsed time 54.37 sec
IALSRecommenderLinear: Epoch 14 of 15. Elapsed time 59.00 sec
IALSRecommenderLinear: Epoch 15 of 15. Elapsed time 1.05 min
IALSRecommenderLinear: Terminating at epoch 15. Elapsed time 1.05 min
|  3        |  0.2301   |  0.3671   |  15.32    |  23.23    |  3.718    |
IALSRecommenderLinear: Epoch 1 of 13. Elapsed time 8.59 sec
IALSRecommenderLinear: Epoch 2 of 13. Elapsed time 17.01 sec
IALSRecommenderLinear: Epoch 3 of 13. Elapsed time 25.35 sec
IALSRecommenderLinear: Epoch 4 of 13. Elapsed time 33.79 sec
IALSRecommenderLinear: Epoch 5 of 13. Elapsed time 42.20 sec
IALSRecommenderLinear: Epoch 6 of 13. Elapsed time 50.65 sec
IALSRecommenderLinear: Epoch 7 of 13. Elapsed time 59.14 se

...

IALSRecommenderLinear: Epoch 31 of 58. Elapsed time 3.88 min
IALSRecommenderLinear: Epoch 32 of 58. Elapsed time 4.01 min
IALSRecommenderLinear: Epoch 33 of 58. Elapsed time 4.13 min
IALSRecommenderLinear: Epoch 34 of 58. Elapsed time 4.25 min
IALSRecommenderLinear: Epoch 35 of 58. Elapsed time 4.37 min
IALSRecommenderLinear: Epoch 36 of 58. Elapsed time 4.50 min
IALSRecommenderLinear: Epoch 37 of 58. Elapsed time 4.62 min
IALSRecommenderLinear: Epoch 38 of 58. Elapsed time 4.74 min
IALSRecommenderLinear: Epoch 39 of 58. Elapsed time 4.87 min
IALSRecommenderLinear: Epoch 40 of 58. Elapsed time 5.03 min
IALSRecommenderLinear: Epoch 41 of 58. Elapsed time 5.16 min
IALSRecommenderLinear: Epoch 42 of 58. Elapsed time 5.28 min
IALSRecommenderLinear: Epoch 43 of 58. Elapsed time 5.40 min
IALSRecommenderLinear: Epoch 44 of 58. Elapsed time 5.52 min
IALSRecommenderLinear: Epoch 45 of 58. Elapsed time 5.64 min
IALSRecommenderLinear: Epoch 46 of 58. Elapsed time 5.77 min
IALSRecommenderLinear: E

IALSRecommenderLinear: Epoch 3 of 41. Elapsed time 34.95 sec
IALSRecommenderLinear: Epoch 4 of 41. Elapsed time 46.27 sec
IALSRecommenderLinear: Epoch 5 of 41. Elapsed time 1.03 min
IALSRecommenderLinear: Epoch 6 of 41. Elapsed time 1.74 min
IALSRecommenderLinear: Epoch 7 of 41. Elapsed time 2.30 min
IALSRecommenderLinear: Epoch 8 of 41. Elapsed time 2.76 min
IALSRecommenderLinear: Epoch 9 of 41. Elapsed time 3.02 min
IALSRecommenderLinear: Epoch 10 of 41. Elapsed time 3.30 min
IALSRecommenderLinear: Epoch 11 of 41. Elapsed time 3.58 min
IALSRecommenderLinear: Epoch 12 of 41. Elapsed time 3.91 min
IALSRecommenderLinear: Epoch 13 of 41. Elapsed time 4.25 min
IALSRecommenderLinear: Epoch 14 of 41. Elapsed time 4.54 min
IALSRecommenderLinear: Epoch 15 of 41. Elapsed time 4.89 min
IALSRecommenderLinear: Epoch 16 of 41. Elapsed time 5.13 min
IALSRecommenderLinear: Epoch 17 of 41. Elapsed time 5.36 min
IALSRecommenderLinear: Epoch 18 of 41. Elapsed time 5.62 min
IALSRecommenderLinear: Epoch 

IALSRecommenderLinear: Epoch 11 of 41. Elapsed time 7.07 min
IALSRecommenderLinear: Epoch 12 of 41. Elapsed time 7.52 min
IALSRecommenderLinear: Epoch 13 of 41. Elapsed time 7.97 min
IALSRecommenderLinear: Epoch 14 of 41. Elapsed time 8.43 min
IALSRecommenderLinear: Epoch 15 of 41. Elapsed time 8.90 min
IALSRecommenderLinear: Epoch 16 of 41. Elapsed time 9.35 min
IALSRecommenderLinear: Epoch 17 of 41. Elapsed time 9.82 min
IALSRecommenderLinear: Epoch 18 of 41. Elapsed time 10.19 min
IALSRecommenderLinear: Epoch 19 of 41. Elapsed time 10.54 min
IALSRecommenderLinear: Epoch 20 of 41. Elapsed time 10.86 min
IALSRecommenderLinear: Epoch 21 of 41. Elapsed time 11.27 min
IALSRecommenderLinear: Epoch 22 of 41. Elapsed time 11.64 min
IALSRecommenderLinear: Epoch 23 of 41. Elapsed time 11.96 min
IALSRecommenderLinear: Epoch 24 of 41. Elapsed time 12.32 min
IALSRecommenderLinear: Epoch 25 of 41. Elapsed time 12.68 min
IALSRecommenderLinear: Epoch 26 of 41. Elapsed time 13.02 min
IALSRecommender

IALSRecommenderLinear: Epoch 5 of 31. Elapsed time 58.15 sec
IALSRecommenderLinear: Epoch 6 of 31. Elapsed time 1.15 min
IALSRecommenderLinear: Epoch 7 of 31. Elapsed time 1.34 min
IALSRecommenderLinear: Epoch 8 of 31. Elapsed time 1.53 min
IALSRecommenderLinear: Epoch 9 of 31. Elapsed time 1.74 min
IALSRecommenderLinear: Epoch 10 of 31. Elapsed time 1.98 min
IALSRecommenderLinear: Epoch 11 of 31. Elapsed time 2.22 min
IALSRecommenderLinear: Epoch 12 of 31. Elapsed time 2.40 min
IALSRecommenderLinear: Epoch 13 of 31. Elapsed time 2.65 min
IALSRecommenderLinear: Epoch 14 of 31. Elapsed time 2.92 min
IALSRecommenderLinear: Epoch 15 of 31. Elapsed time 3.21 min
IALSRecommenderLinear: Epoch 16 of 31. Elapsed time 3.43 min
IALSRecommenderLinear: Epoch 17 of 31. Elapsed time 3.65 min
IALSRecommenderLinear: Epoch 18 of 31. Elapsed time 3.84 min
IALSRecommenderLinear: Epoch 19 of 31. Elapsed time 4.04 min
IALSRecommenderLinear: Epoch 20 of 31. Elapsed time 4.29 min
IALSRecommenderLinear: Epoch

KeyboardInterrupt: 

In [22]:
import json
with open("tuning_KFOLD/IALS/" + IALS_recommenders[0].RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
    json.dump(optimizer.max, json_file)

In [ ]:
# target   |   alpha   |  epochs   | num_fa... |    reg    
#0.2342   |  0.739    |  43.59    |  50.69    |  4.874